# FL Studio Tuturials on Youtube

## Getting started

In [ ]:
# Run this cell once, this will install the API client.
pip install google-api-python-client

#### Once installed, restart and clean the kernel and continue from here:

In [1]:
from googleapiclient.discovery import build
import csv
import time
import os
import json
import pandas as pd
pd.set_option('precision', 0)

#### Create an api_key and store this credential as a Environment Variable on your local device
In this project, the variable is defined in Terminal as  "YOUTUBE_API". For more information, check [this page](https://tilburgsciencehub.com/building-blocks/store-and-document-your-data/store-data/environment-variables/) or [this instructional video](https://www.youtube.com/watch?v=5iWhQWVXosU).

In [2]:
api_key = os.environ["YOUTUBE_API"]

In [3]:
youtube = build('youtube', 'v3', developerKey= api_key)

#### Testing the systems
The cells below will check whether the API functions. 

Within this document, each definition of a parameter, operation or function is explained within the cells. These definitions should be considered as the same throughout the project unless mentioned otherwise.

In [4]:
# check the default number of results, this should give 5

# .search = executes the search method 
# .list = retrieves a list of zero or more resources
# q = query term
# part = identifies group of properties that should be returned
# type = type of resource
# snippet = provides overview with information about the video such as titles, description, thumbnails and tags
# .execute = executes the request

request = youtube.search().list(
            q='FL tutorial',
            part='snippet',
            type='video')
response = request.execute()
print('Total items: ' , len(response['items']))

Total items:  5


In [5]:
# check the maximum number of results, this should give 50

# maxResults = specifies the number of items that should be returned with a maximum of 50

request = youtube.search().list(q='FL tutorial',part='snippet',type='video',maxResults=50)
response = request.execute()
print('Total items: ' , len(response['items']))

Total items:  50


If the two cells above gave you the outputs **5** and **50** respectfully, the API works correctly.

#### Gathering the data

In [6]:
# creates basic for data collection

# 
#
#
# next_page = every page contains a nextPageToken and this is used to iterate over multiple pages


no_requests = 0
max_requests = 100

search_res = []

while no_requests <= max_requests:
    try:
        no_requests += 1
        if (no_requests==1): 
            request = youtube.search().list(q='FL tutorial',part='snippet',type='video',maxResults=50)
        else:
            request = youtube.search().list(q='FL tutorial',part='snippet',type='video',maxResults=50, pageToken = next_page)

        response = request.execute()
        next_page = response['nextPageToken']
    except:    
        break
    for item in response['items']:
        search_res.append(item['snippet'])
    time.sleep(2)

In [7]:
# makes a list of videoIDs

videoIDs = []

for item in search_res:
    videoIDs.append(item['thumbnails']['default']['url'][23:34])

print("Found " + str(len(videoIDs)) + " video IDs!")

Found 600 video IDs!


In [8]:
# --- start of gathering statistics --- #
cnt=0
for vid in videoIDs:
    cnt+=1
    if (cnt==3): 
        break   # overview for only 3 results, just to give impression
    stats = youtube.videos().list(part='statistics',id=vid)

In [9]:
# response for stats 
response_stats = []

for vid in videoIDs:
    stats = youtube.videos().list(part='statistics',id=vid)
    response_stats.append(stats.execute())
    res_stats = {}
    
    for item in response_stats:
        stats = item['items'][0]['statistics']
        res_stats[item['items'][0]['id']] = stats

In [10]:
# output stats to json file
import json
converted_to_string = json.dumps(res_stats)
f = open('stats_output.json', 'a', encoding='utf-8')
f.write(converted_to_string + '\n')
f.close()

In [11]:
# read json file for stats
f = open('stats_output.json', 'r', encoding='utf-8')
content = f.readlines()
for item in content:
    jsonobj = json.loads(item)
    f.close()

In [39]:
# write df for stats and convert to csv
dfstats = pd.read_json(r'stats_output.json', orient = 'index')
#dfstats.to_csv('video_statistics.csv', index_label ='id')

In [41]:
# drops favoriteCount column, this is an old feature that does not longer exist on YouTube

dfstats.drop(dfstats.columns[3],axis=1,inplace=True)

,viewCount,likeCount,dislikeCount,commentCount
pDIsEZsalAo,7291607,203813,2237,7001
Lc4SHa-O4T0,1406260,66629,649,2540
jYBjcEVTSB8,294957,14661,136,722
x5-GweOYXtk,361492,8440,103,341
69Vpk3VKXBE,1546009,29824,1030,921
...,...,...,...,...
qjArVukPOQo,412,39,0,6
VONBCPYE5GE,1,1,0,0
bzHwVr9D0Gc,54,5,0,1
kabRCLkQgfA,165,10,1,3


In [ ]:
# ratio 
comments vs views
#likes vs (likes + dislikes)

,viewCount,likeCount,dislikeCount,commentCount
pDIsEZsalAo,7291607,203813,2237,7001
Lc4SHa-O4T0,1406260,66629,649,2540
jYBjcEVTSB8,294957,14661,136,722
x5-GweOYXtk,361492,8440,103,341
69Vpk3VKXBE,1546009,29824,1030,921
...,...,...,...,...
qjArVukPOQo,412,39,0,6
VONBCPYE5GE,1,1,0,0
bzHwVr9D0Gc,54,5,0,1
kabRCLkQgfA,165,10,1,3


In [ ]:
dfstats.to_csv('video_statistics.csv', index_label ='id')

In [24]:
# --- start of gathering snippets --- #
cnt=0
for vid in videoIDs:
    cnt+=1
    if (cnt==3): break
    snippets = youtube.videos().list(part='snippet',id=vid)

In [25]:
# response for snippet
response_snippets = []

for item in videoIDs:
    snippets = youtube.videos().list(part='snippet',id=item)
    response_snippets.append(snippets.execute())
    res_snippets = {}
    
    for item in response_snippets:
        snippets = item['items'][0]['snippet'] 
        res_snippets[item['items'][0]['id']] = snippets

In [26]:
# output snippets to json file

converted_to_string = json.dumps(res_snippets)
f = open('snippet_output.json', 'a', encoding='utf-8')
f.write(converted_to_string + '\n')
f.close()

In [27]:
# read json file for snippet
f = open('snippet_output.json', 'r', encoding='utf-8')
content = f.readlines()
for item in content:
    jsonobj = json.loads(item)
    f.close()

In [28]:
# write df for snippet to csv
dfsnip = pd.read_json(r'snippet_output.json', orient = 'index')
#dfsnip.to_csv('video_snippets.csv', index_label ='id')

In [29]:
dfsnip #vgm kan/hoeft er niks aangepast worden in dfsnip
#livebroadcast kan er uit


,publishedAt,channelId,title,description,thumbnails,channelTitle,tags,categoryId,liveBroadcastContent,localized,defaultAudioLanguage,defaultLanguage
pDIsEZsalAo,2018-05-23T15:33:40Z,UCIcCXe3iWo6lq-iWKV40Oug,FL Studio 20 - Complete Beginner Basics Tuto...,This video is the first in the FL Studio 20 Ba...,{'default': {'url': 'https://i.ytimg.com/vi/pD...,In The Mix,"[FL Studio 20, FL Studio 20 begginer tutorial,...",26,none,{'title': 'FL Studio 20 - Complete Beginner ...,en-GB,NaN
Lc4SHa-O4T0,2020-05-28T18:00:09Z,UC4Oy0UtQpTBGX46q_OkWzXg,FL STUDIO 20 BEGINNER BEAT MAKING TUTORIAL 202...,FL STUDIO 20 BEGINNER BEAT MAKING TUTORIAL 202...,{'default': {'url': 'https://i.ytimg.com/vi/Lc...,Chuki Beats II,"[making a beat in fl studio, making a beat, ma...",26,none,{'title': 'FL STUDIO 20 BEGINNER BEAT MAKING T...,en,en
jYBjcEVTSB8,2020-12-25T16:21:45Z,UC4Oy0UtQpTBGX46q_OkWzXg,FL STUDIO BEAT MAKING TUTORIAL FOR BEGINNERS 2...,"'Secret Sauce' , Episode 1 - How to make beats...",{'default': {'url': 'https://i.ytimg.com/vi/jY...,Chuki Beats II,"[making a beat in fl studio, making a beat, ma...",26,none,{'title': 'FL STUDIO BEAT MAKING TUTORIAL FOR ...,en,en
x5-GweOYXtk,2020-01-29T09:18:58Z,UCIUtwq6jMFYQj2ASOj95TGw,FL Studio 20 Tutorial | Full FL Studio Crash C...,✅Click Below to get our full Complete FL Studi...,{'default': {'url': 'https://i.ytimg.com/vi/x5...,Tomas George,NaN,27,none,{'title': 'FL Studio 20 Tutorial | Full FL Stu...,en-GB,NaN
69Vpk3VKXBE,2019-06-12T15:57:43Z,UC1kdsZUTDSW1BtsHEoYx4WQ,How to Make a Song in FL Studio 20 💻 | Softwar...,More videos like this FL Studio lesson 👉 https...,{'default': {'url': 'https://i.ytimg.com/vi/69...,Sweetwater,"[fl studio, fl studio tutorial, fl studio 20, ...",10,none,{'title': 'How to Make a Song in FL Studio 20 ...,en,en
...,...,...,...,...,...,...,...,...,...,...,...,...
qjArVukPOQo,2021-11-25T00:37:52Z,UCJ0sIAYr-4oy6U8N3CGCKSg,Buju - Testimony - Remake/Tutorial,[Checkout My Soundkits]\n\nRemake Pack vol1\nh...,{'default': {'url': 'https://i.ytimg.com/vi/qj...,POJbeatz,NaN,10,none,{'title': 'Buju - Testimony - Remake/Tutorial'...,NaN,NaN
VONBCPYE5GE,2021-11-25T18:10:05Z,UCgv3j_g6yhWhuw1qHDfOqTw,Travis Scott Type Beat In 1min Fl Studio Tutorial,#Shorts \nSocial Media:\nPeep My Email : Abdel...,{'default': {'url': 'https://i.ytimg.com/vi/VO...,S1ck Beats,NaN,10,none,{'title': 'Travis Scott Type Beat In 1min Fl S...,NaN,NaN
bzHwVr9D0Gc,2021-11-25T02:58:34Z,UCe1ZcXa-vUWV3FqL9ouBHEQ,How to Add song in FL studio mobile || fl stud...,How to Add song in FL studio\nFL studio mixing...,{'default': {'url': 'https://i.ytimg.com/vi/bz...,TIMLI REMIX,NaN,10,none,{'title': 'How to Add song in FL studio mobile...,NaN,NaN
kabRCLkQgfA,2021-11-24T11:59:37Z,UCbJ6U16IWGjKgbJKNqRmlKg,How to Produce for Ami Faku |FL Studio Tutorial,,{'default': {'url': 'https://i.ytimg.com/vi/ka...,Wayno,NaN,10,none,{'title': 'How to Produce for Ami Faku |FL Stu...,NaN,NaN


In [ ]:
#--- creation of Channel ID list ---#
ChannelIDs = []

for item in search_res:
    ChannelIDs.append(item['channelId'])

In [ ]:
# Response channel

response_channel = []
res_channel = []
res_channels = {}
for chn in ChannelIDs:
    channel = youtube.channels().list(part='statistics',id=chn)
    response_channel.append(channel.execute())
    
    for chn in response_channel:
        channelstat = chn['items'][0]['statistics']
        res_channels[chn['items'][0]['id']] = channelstat

In [ ]:
# output channel to json file

converted_to_string = json.dumps(res_channels)
f = open('channels_output.json', 'a', encoding='utf-8')
f.write(converted_to_string + '\n')
f.close()

In [ ]:
# read json file for channel
f = open('channels_output.json', 'r', encoding='utf-8')
content = f.readlines()
for item in content:
    jsonobj = json.loads(item)
    f.close()

In [ ]:
# write df for channel to csv
dfchn = pd.read_json(r'channels_output.json', orient = 'index')
dfchn.to_csv('video_channels.csv', index_label ='channelId')

In [ ]:
#--- merging the video snippets and statistics --- #
dfsnip = pd.read_csv('video_snippets.csv')
dfstats = pd.read_csv('video_statistics.csv')

dfmerged = dfsnip.merge(dfstats, on='id')
dfmerged.to_csv('video_output.csv')

In [ ]:
# to do list
# functies inbouwen voor ratios maken
# favorite count deleten             ------ check
# kopjes in tabelen verbeteren